In [1]:
# This notebook should be run on GPU in COLAB

In [2]:
# TensorFlow and tf.keras
import tensorflow as tf
import tensorflow_datasets as tfds

import time
import sys 
import os
import numpy as np
import pandas as pd
import gc

import tensorflow.keras.layers as layers

from tensorflow.python.client import device_lib

print(tf.__version__)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

print(device_lib.list_local_devices())

2.6.0
Num GPUs Available:  1
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1016088157258692521
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 16185556992
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7139022745565305297
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [3]:
epochs=3
train_batch_size= 256 # 128 # 64 # 32
val_batch_size=256

save_model=False

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    if save_model:
        from google.colab import drive
        drive.mount('/content/gdrive')

In [4]:
pip install tensorflow_addons transformers tensorflow_text sentencepiece py7zr

In [5]:
!wget -O steam_data.7z 'https://github.com/thacio/Steam-Tags-Tensorflow/raw/main/steam_data.7z'

--2021-11-03 20:33:49--  https://github.com/thacio/Steam-Tags-Tensorflow/raw/main/steam_data.7z
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thacio/Steam-Tags-Tensorflow/main/steam_data.7z [following]
--2021-11-03 20:33:49--  https://raw.githubusercontent.com/thacio/Steam-Tags-Tensorflow/main/steam_data.7z
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24477844 (23M) [application/octet-stream]
Saving to: ‘steam_data.7z’

steam_data.7z       100%[===================>]  23.34M   132MB/s    in 0.2s    

2021-11-03 20:33:49 (132 MB/s) - ‘steam_data.7z’ saved [24477844/24477844]



In [6]:
import py7zr

archive = py7zr.SevenZipFile('steam_data.7z', mode='r')
archive.extractall(path="./")
archive.close()

In [7]:
# Choose transformers model and create tokenizer 

# Create model using transformers
import transformers as transformers
from transformers import AutoTokenizer, TFAutoModel, TFAutoModelForSequenceClassification
import tensorflow as tf

transformer_model_name='distilroberta-base'
# transformer_model_name='bert-base-uncased'
# transformer_model_name='roberta-large'
# transformer_model_name='kamalkraj/deberta-v2-xlarge'

model_config=transformers.PretrainedConfig.from_pretrained(transformer_model_name)
tokenizer = AutoTokenizer.from_pretrained(transformer_model_name)

max_length=model_config.max_position_embeddings
print(model_config)

You are using a model of type roberta to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


PretrainedConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "transformers_version": "4.12.3",
  "type_vocab_size": 1,
  "vocab_size": 50265
}



In [8]:
# load the steam data
steam_data=pd.read_csv('./steam.csv')
steam_description=pd.read_csv('./steam_description_data.csv')
steam_description=steam_description.rename(columns={"steam_appid": "appid"})

steam_data = pd.merge(steam_data,steam_description, on = 'appid')

steam_data = steam_data[steam_data.english == 1]
# Delete the columns we won't use
steam_data = steam_data.drop(['release_date','english','developer','publisher','platforms','required_age','achievements', 'positive_ratings','negative_ratings','average_playtime','median_playtime','owners','price'], 1)
steam_data = steam_data.drop(['about_the_game','short_description'], 1)

In [9]:
from bs4 import BeautifulSoup
import re
import tensorflow_text as tf_text

def clean_text(text):
    text = BeautifulSoup(text).get_text()
    text = re.sub("https*\S+", " ", text)
    text = re.sub("#\S+", " ", text)
    text = re.sub("\\*", " ", text)
    text = re.sub("/\d\.\s+|[a-z]\)\s+|•\s+|[A-Z]\.\s+|[IVX]+\.\s+/g", ".", text)
    text = re.sub("[®●™©]", ".", text)
    re.sub(r'[^\x00-\x7F]+','.', text) # not working?
    text = re.sub("\x093", ".", text)
    text = re.sub("\x096", ".", text)
    text = re.sub("\x097", ".", text)
    text = re.sub("\x099", ".", text)
    text = re.sub("\t+", ".", text)
    text = re.sub("\n\.", "\n", text)
    text = re.sub("\"[\r\n]", "\n", text)
    text = re.sub("\r\n{2,}", "\r\n", text)
    text = re.sub("\.-", ".", text)
    text = re.sub("\.+",".", text)
    text = re.sub("\. \.", ".", text)
    text = re.sub(",\.", ".", text)
    # text = tf_text.normalize_utf8(text, 'NFKD')
    text = re.sub(' +', ' ',text) # remove several spaces
    # text = tf.strings.strip(text)
    return text

steam_data['detailed_description'] = steam_data['detailed_description'].apply(lambda x: clean_text(x))

In [10]:
# https://towardsdatascience.com/cleaning-text-data-with-python-b69b47b97b76
steam_data.head()

,appid,name,categories,genres,steamspy_tags,detailed_description
0,10,Counter-Strike,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,Play the world's number 1 online action game. ...
1,20,Team Fortress Classic,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,One of the most popular online action games of...
2,30,Day of Defeat,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,Enlist in an intense brand of Axis vs. Allied ...
3,40,Deathmatch Classic,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,Enjoy fast-paced multiplayer gaming with Death...
4,50,Half-Life: Opposing Force,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,Return to the Black Mesa Research Facility as ...


In [11]:
# steam_data columns: appid	name, categories, genres, steamspy_tags, detailed_description, about_the_game, short_description

# Get a list of unique values for classification tags
dict_games=dict()
dict_games['categories']=set()
dict_games['genres']=set()
dict_games['steamspy_tags']=set()

# Find all unique tags, categories and genres
for index, row in steam_data.iterrows():
    text=row['categories'].split(';')
    for i in range(len(text)):
        dict_games['categories'].add(text[i])

    text=row['genres'].split(';')
    for i in range(len(text)):
        dict_games['genres'].add(text[i])

    text=row['steamspy_tags'].split(';')
    for i in range(len(text)):
        dict_games['steamspy_tags'].add(text[i])

dict_games['categories']=sorted(list(dict_games['categories']))
dict_games['genres']=sorted(list(dict_games['genres']))
dict_games['steamspy_tags']=sorted(list(dict_games['steamspy_tags']))

In [12]:
# create one-hot vectors from category
def to_one_hot_vectors(text,categories_list):
    x = text.split(';')
    one_hot_vector=np.zeros(len(categories_list))

    for item in x:
        one_hot_vector[categories_list.index(item)]=1

    # return one_hot_vector
    return tf.convert_to_tensor(one_hot_vector)

In [13]:
# Create inputs and outputs for training
x_token_list=list()
x_inputs_ids_list=list()
x_attention_masks_list=list()
y_dict=dict()
y_dict['categories']=list()
y_dict['genres']=list()
y_dict['steamspy_tags']=list()
index_on_steam_data_pd=list()


dot_id=tokenizer.get_vocab()['.']
eos_id=0
pad_id=0

for index, row in steam_data.iterrows():

    # token with truncation and padding for training
    token = tokenizer(row['detailed_description'], truncation=True, padding="max_length",max_length=max_length)    

    was_truncated=(token['input_ids'][-1]!=tokenizer.pad_token_id)

    # If string was truncated, we should find and pad until the last dot pontuaction '.' and add eos_id
    # if not, than we should find last not pad_id and add eos_id right affter it
    if was_truncated:
        input_id=token['input_ids']
        for i in range(len(input_id)-1,0,-1):
                if input_id[i]==dot_id and i+1!=max_length:
                    break
                else:
                    token['input_ids'][i]=pad_id
                    token['attention_mask'][i]=0

    token['input_ids']=tf.expand_dims(tf.convert_to_tensor(token['input_ids']),axis=0)
    token['attention_mask']=tf.expand_dims(tf.convert_to_tensor(token['attention_mask']),axis=0)


    # token = tokenizer(row['detailed_description'], return_tensors="tf",truncation=True, padding="max_length",max_length=max_length)
    # # token of the descriptions for checking description length
    # token = tokenizer(row['detailed_description'], return_tensors="tf")

    # # If the training description is longer than our model input length, remove it from training set
    # if token['input_ids'].shape[1] > max_length - 2:
    #    steam_data.drop(index, inplace=True)
    #    continue

    # # token with truncation and padding for training
    # token = tokenizer(row['detailed_description'], return_tensors="tf",truncation=True, padding="max_length",max_length=max_length)
    

    x_token_list.append(token)
    x_inputs_ids_list.append(token['input_ids'])
    x_attention_masks_list.append(token['attention_mask'])
    index_on_steam_data_pd.append(index)

    # one-hot vectors outputs for classification
    y_dict['categories'].append(to_one_hot_vectors(row['categories'],dict_games['categories']))
    y_dict['genres'].append(to_one_hot_vectors(row['genres'],dict_games['genres']))
    y_dict['steamspy_tags'].append(to_one_hot_vectors(row['steamspy_tags'],dict_games['steamspy_tags']))

In [14]:
steam_data.to_csv('steam_dataset.csv',sep='\t')

In [15]:
import sklearn.model_selection as model_selection

# Split into train and test set
seed=25
x_ids_train, x_ids_test, x_att_train, x_att_test, y_train, y_test, index_on_steam_data_pd_train, index_on_steam_data_pd_test = model_selection.train_test_split(
      x_inputs_ids_list,x_attention_masks_list, y_dict['steamspy_tags'], index_on_steam_data_pd,
      train_size=0.8,test_size=0.2, random_state=seed)

# Create Train and validation datasets
y=tf.convert_to_tensor(y_train)
x_ids=tf.squeeze(tf.convert_to_tensor(x_ids_train))
x_att=tf.squeeze(tf.convert_to_tensor(x_att_train))
ds_train = tf.data.Dataset.from_tensor_slices( ( x_ids, x_att, y) )

y=tf.convert_to_tensor(y_test)
x_ids=tf.squeeze(tf.convert_to_tensor(x_ids_test))
x_att=tf.squeeze(tf.convert_to_tensor(x_att_test))
ds_val = tf.data.Dataset.from_tensor_slices( ( x_ids, x_att, y) )

num_train_examples=len(ds_train)
num_val_examples=len(ds_val)
print('Train dataset count:',num_train_examples)
print('Test dataset count:',num_val_examples)

# batch datasets
ds_train=ds_train.batch(train_batch_size)
ds_val=ds_val.batch(val_batch_size)

num_train_size=len(ds_train)
num_val_size=len(ds_val)

del x_ids, x_att, y

Train dataset count: 21251
Test dataset count: 5313


In [16]:
# ler depois https://datasciencetoday.net/index.php/en-us/nlp/211-paper-dissected-bert-pre-training-of-deep-bidirectional-transformers-for-language-understanding-explained

# Build model using last_hidden_state and pooler output concatenated
def build_model_last_hidden_state(transformer,num_layers,max_length=max_length):
    input_ids = tf.keras.layers.Input(shape=(max_length,),name='input_ids', dtype='int32')
    input_attention_mask = tf.keras.layers.Input(shape=(max_length,),name='input_attention_mask', dtype='int32')  
    
    x = model(input_ids,input_attention_mask)
    x = tf.keras.layers.Concatenate(axis=1)([x[0], tf.expand_dims(x[1],axis=1)]) # concatenate last_hidden_state and pooler_output
    x = tf.keras.layers.Flatten()(x)

    x = tf.keras.layers.Dense(num_layers)(x)
    output = tf.keras.layers.Activation('sigmoid')(x)
    return tf.keras.Model([input_ids,input_attention_mask], output)

# Build model with only pooler_output
# Doesn't seem to get good results with large number of labels, as steamspy_tags (339 total)
def build_model_pooler_output(transformer,num_layers,max_length=max_length):
    input_ids = tf.keras.layers.Input(shape=(max_length,),name='input_ids', dtype='int32')
    input_attention_mask = tf.keras.layers.Input(shape=(max_length,),name='input_attention_mask', dtype='int32')

    # using pooler_output
    pooler_output = model(input_ids,input_attention_mask)[1]
    output = tf.keras.layers.Dense(num_layers, activation='sigmoid')(pooler_output)
    return tf.keras.Model([input_ids,input_attention_mask], output)

In [17]:
# Create model for classification of steamspy_tag
try:
    del model
except:
    None

if 'model' not in vars():
    model = TFAutoModel.from_pretrained(transformer_model_name)

    for layer in model.layers:
        layer.trainable = False

    # model=build_model_pooler_output(model,num_layers=len(dict_games['steamspy_tags']),max_length=max_length)
    model=build_model_last_hidden_state(model,num_layers=len(dict_games['steamspy_tags']),max_length=max_length)

model.summary()

Some layers from the model checkpoint at distilroberta-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at distilroberta-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 514)]        0                                            
__________________________________________________________________________________________________
input_attention_mask (InputLaye [(None, 514)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode TFBaseModelOutputWit 82118400    input_ids[0][0]                  
                                                                 input_attention_mask[0][0]       
__________________________________________________________________________________________________
tf.expand_dims (TFOpLambda)     (None, 1, 768)       0           tf_roberta_model[0][1]       

In [18]:
import tensorflow_addons as tfa

# losses
loss_fn=tf.keras.losses.BinaryCrossentropy()
# loss_fn=tfa.losses.SigmoidFocalCrossEntropy()

optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5)

model.compile(
    optimizer=optimizer,
    loss=loss_fn,
    metrics=tfa.metrics.F1Score(num_classes=len(dict_games['steamspy_tags']),threshold=0.5),
)

In [19]:
@tf.function
def train_step(model,x_batch_train_ids, x_batch_train_attentions, y_batch_train):
    with tf.GradientTape() as tape:
        predictions = model([x_batch_train_ids,x_batch_train_attentions])
        loss = model.loss(y_true=y_batch_train,y_pred=predictions)

    grads = tape.gradient(loss, model.trainable_weights)
    model.optimizer.apply_gradients(zip(grads, model.trainable_weights))

    return loss, predictions

@tf.function
def val_step(model,x_batch_train_ids, x_batch_train_attentions, y_batch_train):
    predictions = model([x_batch_train_ids,x_batch_train_attentions])
    loss = model.loss(y_true=y_batch_train,y_pred=predictions)
    
    return loss, predictions

In [20]:
def top_k_multilabel_accuracy(y_pred,y_true,k):
    # sort true labels and get the number of positive labels
    sorted=tf.argsort(y_true, direction='DESCENDING')
    num_labels=tf.cast(tf.reduce_sum(y_true,axis=1),tf.int32)

    # get top_k predictions
    top_k_pred=tf.math.top_k(y_pred,k=k)

    in_top_k_tf = tf.TensorArray(tf.float32, size=0, dynamic_size=True, clear_after_read=False)

    # count the true positive labels in the predictions top_k
    for i in range(y_true.shape[0]):
        label_indices=sorted[i,:num_labels[i]]
        in_top_k=0.0
        for j in range(len(label_indices)):
            labels_found=tf.reduce_sum( tf.cast( top_k_pred[1][i,:]==label_indices[j],tf.float32) )
            in_top_k=in_top_k + labels_found
        
        in_top_k_tf=in_top_k_tf.write(i,in_top_k)
        None

    # batch number of labels in top k
    in_top_k_tf=in_top_k_tf.stack()

    accuracy=in_top_k_tf/tf.cast(num_labels,tf.float32)

    return accuracy

In [21]:
def best_predictions_descending(predictions,labels_list):
    predictions=tf.squeeze(predictions).numpy()
    labels_list=np.array(labels_list)
    inds = np.argsort(-predictions)
    return list(zip(labels_list[inds],predictions[inds]))

def get_best_game_classifications(description,model,labels_list):
    token = tokenizer(description, return_tensors="tf",truncation=True, padding="max_length",max_length=max_length)
    predictions=tf.squeeze(model([ token['input_ids'], token['attention_mask'] ]) )
    return best_predictions_descending(predictions,labels_list)

def report_stemaspytags_game_classification(index):
    i=index
    description=clean_text(steam_data['detailed_description'][i])
    y_label=to_one_hot_vectors(steam_data['steamspy_tags'][i],dict_games['steamspy_tags'])

    print(steam_data['name'][i])
    print(steam_data['steamspy_tags'][i])
    print('--------Description----------')
    print(description)
    print('-----------------------------')

    tags=get_best_game_classifications(description,model,dict_games['steamspy_tags'])


    ####################
    token = tokenizer(description, return_tensors="tf",truncation=True, padding="max_length",max_length=max_length)
    predictions=tf.squeeze(model([ token['input_ids'], token['attention_mask'] ]) )

    val_accuracy_top10=top_k_multilabel_accuracy(y_pred=tf.expand_dims(predictions,axis=0),y_true=tf.expand_dims(y_label,axis=0),k=10)
    val_accuracy_top5=top_k_multilabel_accuracy(y_pred=tf.expand_dims(predictions,axis=0),y_true=tf.expand_dims(y_label,axis=0),k=5)
    val_accuracy_top3=top_k_multilabel_accuracy(y_pred=tf.expand_dims(predictions,axis=0),y_true=tf.expand_dims(y_label,axis=0),k=3)

    print('predicted tags')
    print(tags)
    print(
        'top',10,f'acc: {val_accuracy_top10.numpy()[0]:.5f}, '
        'top',5,f'acc: {val_accuracy_top5.numpy()[0]:.5f}, '
        'top',3,f'acc: {val_accuracy_top3.numpy()[0]:.5f}, '
        )

In [ ]:
for epoch in range(1,epochs+1):

    ##### Training block ######    
    print('Epoch ',epoch,'/',epochs)
        
    ds_train.shuffle(num_train_examples)

    num_trained_samples=0.0
    train_loss=0.0
    train_acc=0.0
    train_acc_top10=0.0
    train_acc_top5=0.0
    train_acc_top3=0.0
    
    skipped_count=0
    for step, (x_batch_train_ids, x_batch_train_attentions, y_batch_train) in enumerate(ds_train):
 
        try:
            batch_loss, predictions=train_step(model,x_batch_train_ids, x_batch_train_attentions, y_batch_train)
        except:
            # while using TPU or CPU in colab, some examples are causing error. On GPU, everything runs fine
            skipped_count=skipped_count+1            
            print('step ',step+1,' - # of skipped: ',skipped_count,'epoch ',epoch,'- Train step ',int(num_trained_samples),'/',num_train_examples)
            continue
          
        # Some tensorflow loss functions return loss per batch sample instead of sum or mean loss, sum them if that's the case
        if (tf.rank(batch_loss)==1).numpy():
            batch_loss=tf.reduce_sum(batch_loss)
            train_loss += batch_loss.numpy()
        else:
            train_loss += batch_loss.numpy()*y_batch_train.shape[0]

        # Update training metrics
        num_trained_samples += y_batch_train.shape[0]
        
        acc=top_k_multilabel_accuracy(y_pred=predictions,y_true=y_batch_train,k=10)
        train_acc_top10 += tf.reduce_sum(acc)

        acc=top_k_multilabel_accuracy(y_pred=predictions,y_true=y_batch_train,k=5)
        train_acc_top5 += tf.reduce_sum(acc)

        acc=top_k_multilabel_accuracy(y_pred=predictions,y_true=y_batch_train,k=3)
        train_acc_top3 += tf.reduce_sum(acc)

        if step % 10 == 0:
            print('epoch ',epoch,'- Train step ',int(num_trained_samples),'/',num_train_examples,
                  f'last batch loss: {batch_loss.numpy():.5f}, ',
                  f'total mean loss: {train_loss/num_trained_samples:.5f}, ',
                  'top',10,f'acc: {train_acc_top10/num_trained_samples:.5f}, '
                  'top',5,f'acc: {train_acc_top5/num_trained_samples:.5f}, '
                  'top',3,f'acc: {train_acc_top3/num_trained_samples:.5f}, '
                  )  
            
    print('epoch ',epoch,' - Total # of skipped batches: ',skipped_count,'\n')

    print('Total score - epoch ',epoch,'- Train step ',int(num_trained_samples),'/',num_train_examples,
          f'total mean loss: {train_loss/num_trained_samples:.5f}, ',
          'top',10,f'acc: {train_acc_top10/num_trained_samples:.5f}, '
          'top',5,f'acc: {train_acc_top5/num_trained_samples:.5f}, '
          'top',3,f'acc: {train_acc_top3/num_trained_samples:.5f}, '
          )      
    
    ##### Validation block ######       

    # only run after some epochs    
    if epoch % 1 ==0:

        num_val_samples=0.0
        val_loss=0.0
        val_acc=0.0
        val_acc_top10=0.0
        val_acc_top5=0.0
        val_acc_top3=0.0
        
        skipped_count=0
        for step, (x_batch_val_ids, x_batch_val_attentions, y_batch_val) in enumerate(ds_val):
    
            batch_loss, predictions=val_step(model,x_batch_val_ids, x_batch_val_attentions, y_batch_val)
              
            # Some tensorflow loss functions return loss per batch sample instead of sum or mean loss
            if (tf.rank(batch_loss)==1).numpy():
                batch_loss=tf.reduce_sum(batch_loss)
                val_loss += batch_loss.numpy()
            else:
                val_loss += batch_loss.numpy()*y_batch_val.shape[0]

            # Update valing metrics
            num_val_samples += y_batch_val.shape[0]
            
            acc=top_k_multilabel_accuracy(y_pred=predictions,y_true=y_batch_val,k=10)
            val_acc_top10 += tf.reduce_sum(acc)

            acc=top_k_multilabel_accuracy(y_pred=predictions,y_true=y_batch_val,k=5)
            val_acc_top5 += tf.reduce_sum(acc)

            acc=top_k_multilabel_accuracy(y_pred=predictions,y_true=y_batch_val,k=3)
            val_acc_top3 += tf.reduce_sum(acc)

            if step % 10 == 0:
                print('epoch ',epoch,'- val step ',int(num_val_samples),'/',num_val_examples,
                      f'last batch loss: {batch_loss.numpy():.5f}, ',
                      f'total mean loss: {val_loss/num_val_samples:.5f}, ',
                      'top',10,f'acc: {val_acc_top10/num_val_samples:.5f}, '
                      'top',5,f'acc: {val_acc_top5/num_val_samples:.5f}, '
                      'top',3,f'acc: {val_acc_top3/num_val_samples:.5f}, '
                      )  
        print('Total validation score - epoch ',epoch,'- val step ',int(num_val_samples),'/',num_val_examples,
              f'total loss: {val_loss/num_val_samples:.5f}, ',
              'top',10,f'acc: {val_acc_top10/num_val_samples:.5f}, '
              'top',5,f'acc: {val_acc_top5/num_val_samples:.5f}, '
              'top',3,f'acc: {val_acc_top3/num_val_samples:.5f}, '
              )
        print('------------------------------')
        print('---Examples---')
        report_stemaspytags_game_classification(index_on_steam_data_pd_test[30])
        print('\n')
        report_stemaspytags_game_classification(index_on_steam_data_pd_test[100])
        print('\n')
        report_stemaspytags_game_classification(index_on_steam_data_pd_test[200])
        print('\n')
        report_stemaspytags_game_classification(index_on_steam_data_pd_test[400])
        print('\n')
        report_stemaspytags_game_classification(index_on_steam_data_pd_test[4000])

Epoch  1 / 3
epoch  1 - Train step  256 / 21251 last batch loss: 0.75004,  total mean loss: 0.75004,  top 10 acc: 0.04818, top 5 acc: 0.01107, top 3 acc: 0.00391, 
epoch  1 - Train step  2816 / 21251 last batch loss: 0.03368,  total mean loss: 0.11951,  top 10 acc: 0.57037, top 5 acc: 0.47775, top 3 acc: 0.37553, 
epoch  1 - Train step  5376 / 21251 last batch loss: 0.03136,  total mean loss: 0.07778,  top 10 acc: 0.65309, top 5 acc: 0.54905, top 3 acc: 0.44293, 
epoch  1 - Train step  7936 / 21251 last batch loss: 0.03040,  total mean loss: 0.06233,  top 10 acc: 0.69218, top 5 acc: 0.58060, top 3 acc: 0.47350, 


In [ ]:
# for i in range(len(index_on_steam_data_pd_test)):
#     print(i,' - ',steam_data['name'][index_on_steam_data_pd_test[i]])